<br>

<font size=6 color='lightblue'>**[TUTORIAL] MLFLow**</font>

---
---

In [3]:
from platform import platform, python_version

print(f'Platform: {platform()}')
print(f'Python Version: {python_version()}')

Platform: Linux-5.15.0-170-generic-x86_64-with-glibc2.35
Python Version: 3.13.5
